In [281]:
import definitions as d
import neural_network as nn
import numpy as np

from test_case_creator import (
    denormalized,
    get_sets__without_neighbors__one_prediction__without_aggregation__without_mid_prediction,
    get_sets__without_neighbors__one_prediction__with_aggregation__without_mid_prediction
)

In [269]:
rng = np.random.default_rng(1)

def get_nn(layer_sizes, activations, loss):
    assert len(layer_sizes) == len(activations)

    d1_layer = nn.InputLayer(120, "d1")
    d2_layer = nn.InputLayer(120, "d2")
    d3_layer = nn.InputLayer(120, "d3")
    days_layer = nn.MergeLayer([d1_layer, d2_layer, d3_layer])

    coh_layer = nn.InputLayer(36, "city_one_hot")
    date_layer = nn.InputLayer(1, "date")
    cord_layer = nn.InputLayer(2, "cord")
    city_layer = nn.MergeLayer([coh_layer, date_layer, cord_layer])

    output_layer = nn.MergeLayer([days_layer, city_layer])
    for (n, activation) in zip(layer_sizes, activations):
        output_layer = nn.FullConnectLayer(output_layer, n, activation, rng)
    return nn.NeuralNetwork(output_layer, loss)

def get_nn2(layer_sizes, activations, loss):
    assert len(layer_sizes) == len(activations)

    d1_layer = nn.InputLayer(120, "d1")
    d2_layer = nn.InputLayer(120, "d2")
    d3_layer = nn.InputLayer(120, "d3")
    # coh_layer = nn.InputLayer(36, "city_one_hot")
    # date_layer = nn.InputLayer(1, "date")
    # cord_layer = nn.InputLayer(2, "cord")

    output_layer = nn.MergeLayer([d1_layer, d2_layer, d3_layer])
    for (n, activation) in zip(layer_sizes, activations):
        output_layer = nn.FullConnectLayer(output_layer, n, activation, rng)
    return nn.NeuralNetwork(output_layer, loss)

In [440]:
(train_set, test_set, params) = get_sets__without_neighbors__one_prediction__without_aggregation__without_mid_prediction()
# (train_set, test_set, params) = get_sets__without_neighbors__one_prediction__with_aggregation__without_mid_prediction()

In [367]:
rng = np.random.default_rng(1)

def get_day_layer(num):
    l = nn.InputLayer(120, f"d{num}")
    return nn.FullConnectLayer(l, 60, d.linear, rng)

def get_days_layer():
    ls = [get_day_layer(1), get_day_layer(2), get_day_layer(3)]
    l = nn.MergeLayer(ls)
    return nn.FullConnectLayer(l, 100, d.linear, rng)

def get_city_layer():
    coh = nn.InputLayer(36, "city_one_hot")
    date = nn.InputLayer(1, "date")
    cord = nn.InputLayer(2, "cord")
    l = nn.MergeLayer([coh, date, cord])
    return nn.FullConnectLayer(l, 20, d.linear, rng)

def get_nn1(layer_sizes, activations, loss):
    assert len(layer_sizes) == len(activations)

    ds = get_days_layer()
    c = get_city_layer()
    l = nn.MergeLayer([ds, c])
    for (n, activation) in zip(layer_sizes, activations):
        l = nn.FullConnectLayer(l, n, activation, rng)
    return nn.NeuralNetwork(l, loss)

In [417]:
rng = np.random.default_rng(1)

def get_days_layer():
    ld1 = nn.InputLayer(120, "d1")
    ld2 = nn.InputLayer(120, "d2")
    ld3 = nn.InputLayer(120, "d3")

    ld11 = nn.FullConnectLayer(ld1, 60, d.linear, rng)
    ld22 = nn.FullConnectLayer(ld2, 60, d.linear, rng, ld11)
    ld33 = nn.FullConnectLayer(ld3, 60, d.linear, rng, ld11)

    l = nn.MergeLayer([ld11, ld22, ld33])
    return nn.FullConnectLayer(l, 100, d.linear, rng)

def get_city_layer():
    coh = nn.InputLayer(36, "city_one_hot")
    date = nn.InputLayer(1, "date")
    cord = nn.InputLayer(2, "cord")
    l = nn.MergeLayer([coh, date, cord])
    return nn.FullConnectLayer(l, 20, d.linear, rng)

def get_nn3(layer_sizes, activations, loss):
    assert len(layer_sizes) == len(activations)

    ds = get_days_layer()
    c = get_city_layer()
    l = nn.MergeLayer([ds, c])
    for (n, activation) in zip(layer_sizes, activations):
        l = nn.FullConnectLayer(l, n, activation, rng)
    return nn.NeuralNetwork(l, loss)

In [ ]:
# best ones for get_sets__without_neighbors__one_prediction__without_aggregation__without_mid_prediction:
net3 = get_nn([300, 100, 60, 1, 1], [d.relu, d.relu, d.relu, d.relu, d.linear], d.l2_loss)  # ~15.74%
net3.train(train_set, test_set, 1024, "output_temp", rng, 15)

net3 = get_nn3([300, 100, 60, 1, 1], [d.relu, d.relu, d.relu, d.relu, d.linear], d.l2_loss)  # ~15.77% shared weights for days
net3.train(train_set, test_set, 1024, "output_temp", rng, 15)

net3 = get_nn([300, 100, 60, 1, 1], [d.sigmoid, d.sigmoid, d.sigmoid, d.sigmoid, d.linear], d.l2_loss)  # ~15.91%
net3.train(train_set, test_set, 1024, "output_temp", rng, 5)

net3 = get_nn([300, 100, 60, 1, 1], [d.sigmoid, d.sigmoid, d.sigmoid, d.sigmoid, d.linear], d.l1_loss)  # ~16.13%
net3.train(train_set, test_set, 1024, "output_temp", rng, 5)

net3 = get_nn([300, 100, 60, 1, 1], [d.sigmoid, d.sigmoid, d.sigmoid, d.sigmoid, d.linear], d.l1_loss)  # ~16.29% shared weights for days
net3.train(train_set, test_set, 1024, "output_temp", rng, 5)

net3 = get_nn1([100, 40, 1], [d.linear, d.linear, d.linear], d.l2_loss)  # ~18.17%
net3.train(train_set, test_set, 1024, "output_temp", rng, 5)

net3_wind = get_nn2([80, 40, 2, 2, 2], [d.linear, d.linear, d.linear, d.sigmoid, d.softmax], d.cross_entropy_loss)  # 61.49% no matter hinge or cross entropy
net3_wind.train(train_set, test_set, 1024, "output_wind", rng, 5)

net3_wind = get_nn2([80, 40, 2, 2, 2], [d.linear, d.linear, d.linear, d.sigmoid, d.softmax], d.cross_entropy_loss)  # 61.49% no matter hinge or cross entropy shared weights for days
net3_wind.train(train_set, test_set, 1024, "output_wind", rng, 5)


# best ones for get_sets__without_neighbors__one_prediction__with_aggregation__without_mid_prediction:
net3 = get_nn1([100, 40, 1], [d.linear, d.linear, d.linear], d.l2_loss)  # ~12.77%
net3.train(train_set, test_set, 1024, "output_temp", rng, 5)

net3 = get_nn([300, 100, 60, 1, 1], [d.sigmoid, d.sigmoid, d.sigmoid, d.sigmoid, d.linear], d.l1_loss)  # ~16.99%
net3.train(train_set, test_set, 1024, "output_temp", rng, 5)

net3 = get_nn([300, 100, 60, 1, 1], [d.sigmoid, d.sigmoid, d.sigmoid, d.sigmoid, d.linear], d.l1_loss)  # ~17.51% shared weights for days
net3.train(train_set, test_set, 1024, "output_temp", rng, 5)

net3_wind = get_nn2([80, 40, 2, 2, 2], [d.linear, d.linear, d.linear, d.sigmoid, d.softmax], d.cross_entropy_loss)  # 53.18% no matter hinge or cross entropy
net3_wind.train(train_set, test_set, 1024, "output_wind", rng, 5)

net3_wind = get_nn2([80, 40, 2, 2, 2], [d.linear, d.linear, d.linear, d.sigmoid, d.softmax], d.cross_entropy_loss)  # 53.18% no matter hinge or cross entropy shared weights for days
net3_wind.train(train_set, test_set, 1024, "output_wind", rng, 5)

In [445]:
net3 = get_nn3([300, 100, 60, 1, 1], [d.relu, d.relu, d.relu, d.relu, d.linear], d.l2_loss)  # ~15.31% shared weights for days
net3.train(train_set, test_set, 1024, "output_temp", rng, 15)

Epoch 1/15: train: 0.017677084673304203, test: 0.014369078538024316
Epoch 2/15: train: 0.01445565768162976, test: 0.014365033828265568
Epoch 3/15: train: 0.014439904289636644, test: 0.014369319849455059
Epoch 4/15: train: 0.014439542840870566, test: 0.014373736423082022
Epoch 5/15: train: 0.014439515799399977, test: 0.014377153908846939
Epoch 6/15: train: 0.014439732544360146, test: 0.014379642008586959
Epoch 7/15: train: 0.014440083300858823, test: 0.014381617969476169
Epoch 8/15: train: 0.014440513547725723, test: 0.014383135460859217
Epoch 9/15: train: 0.014440995535915765, test: 0.01438414836311623
Epoch 10/15: train: 0.014441504635707628, test: 0.014384559390870475
Epoch 11/15: train: 0.01444201409640495, test: 0.014384266878245873
Epoch 12/15: train: 0.014442493628142322, test: 0.014383211707243125
Epoch 13/15: train: 0.014442911313133733, test: 0.014381420949482615
Epoch 14/15: train: 0.014443238062569672, test: 0.014379031948331013
Epoch 15/15: train: 0.01444345329781733, test:

In [446]:
predicted = net3.predict(train_set)
expected = train_set["output_temp"]

print(predicted)
print(expected)

predicted = denormalized(predicted, params["temperature"])
expected = denormalized(expected, params["temperature"])

print(predicted)
print(expected)

diffs = np.abs(predicted - expected)
print(f"min: {np.min(diffs)}, max: {np.max(diffs)}, mean: {np.mean(diffs)}, median: {np.median(diffs)}")
print(f"Good predictions: {np.count_nonzero(diffs <= 2)}, bad predictions: {np.count_nonzero(diffs > 2)}, success rate: {np.count_nonzero(diffs <= 2) / diffs.size * 100 : .2f}%")

predicted = net3.predict(test_set)
expected = test_set["output_temp"]

print(predicted)
print(expected)

predicted = denormalized(predicted, params["temperature"])
expected = denormalized(expected, params["temperature"])

print(predicted)
print(expected)

diffs = np.abs(predicted - expected)
print(f"min: {np.min(diffs)}, max: {np.max(diffs)}, mean: {np.mean(diffs)}, median: {np.median(diffs)}")
print(f"Good predictions: {np.count_nonzero(diffs <= 2)}, bad predictions: {np.count_nonzero(diffs > 2)}, success rate: {np.count_nonzero(diffs <= 2) / diffs.size * 100 : .2f}%")

print(np.average(d.l2_loss_n(predicted, expected)))

[[0.59839751 0.59839751 0.59839751 ... 0.59839751 0.59839751 0.59839751]]
[[0.5753539  0.57437143 0.57095394 ... 0.64950349 0.61789978 0.57858124]]
[[289.5402573 289.5402573 289.5402573 ... 289.5402573 289.5402573
  289.5402573]]
[[287.7225     287.645      287.37541667 ... 293.57166667 291.07866108
  287.97708333]]
min: 0.0001593660670096142, max: 41.077444800599636, mean: 7.5016760396423905, median: 6.459381201766291
Good predictions: 7393, bad predictions: 38290, success rate:  16.18%
[[0.59839751 0.59839751 0.59839751 ... 0.59839751 0.59839751 0.59839751]]
[[0.39678851 0.36225967 0.31486372 ... 0.67760934 0.68179801 0.69291554]]
[[289.5402573 289.5402573 289.5402573 ... 289.5402573 289.5402573
  289.5402573]]
[[273.63666667 270.91291667 267.17416667 ... 295.78875    296.11916667
  296.99615385]]
min: 0.0005760327337043236, max: 38.56240664376631, mean: 7.613466083581378, median: 6.678909366066989
Good predictions: 1957, bad predictions: 10454, success rate:  15.77%
89.4575570420064

In [447]:
net3_wind = get_nn2([80, 40, 2, 2, 2], [d.linear, d.linear, d.linear, d.sigmoid, d.softmax], d.cross_entropy_loss)  # 61.49% no matter hinge or cross entropy
net3_wind.train(train_set, test_set, 1024, "output_wind", rng, 5)

Epoch 1/5: train: 0.5667184855836179, test: 0.7064634393468613
Epoch 2/5: train: 0.5567351869560997, test: 0.715061707880622
Epoch 3/5: train: 0.556772205210926, test: 0.7151248616412659
Epoch 4/5: train: 0.5567751819507388, test: 0.714116552352304
Epoch 5/5: train: 0.5567698392420339, test: 0.7131947541093019


In [448]:
predicted = net3_wind.predict(test_set)
print(predicted)
print(np.max(predicted, axis=1))
predicted = np.rint(predicted[0, :])
expected = test_set["output_wind"][0, :]
print(predicted)
print(expected)
print(np.count_nonzero(predicted == 1))
print(predicted.size)
print(f"Good predictions: {np.count_nonzero(predicted == expected)}, bad predictions: {np.count_nonzero(predicted != expected)}, success_rate: {np.count_nonzero(predicted == expected) / predicted.size * 100 : .2f}%")

[[0.24673702 0.24673702 0.24673702 ... 0.24673702 0.24673702 0.24673702]
 [0.75326298 0.75326298 0.75326298 ... 0.75326298 0.75326298 0.75326298]]
[0.24673712 0.75326298]
[0. 0. 0. ... 0. 0. 0.]
[1 0 0 ... 0 1 0]
0
12411
Good predictions: 7631, bad predictions: 4780, success_rate:  61.49%
